In [40]:
import os
import json
import numpy as np
from tifffile import imread
import cv2
import skimage.io as sio

import albumentations as A
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection import MaskRCNN, FasterRCNN_ResNet50_FPN_Weights, MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models import ResNet50_Weights
from torchvision.ops import box_convert
import torchvision.transforms as T
import torch.nn.functional as F

import torch
from torch.optim import SGD, lr_scheduler

import pathlib
import json

from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

from PIL import Image

In [14]:
!gdown https://drive.google.com/file/d/1B0qWNzQZQmfQP7x7o4FDdgb9GvPDoFzI/view --fuzzy
!mkdir ../dataset
!tar -xzf hw3-data-release.tar.gz
!mv test_release/ ../dataset
!mv train/ ../dataset/
!mv test_image_name_to_ids.json ../dataset/

Downloading...
From (original): https://drive.google.com/uc?id=1B0qWNzQZQmfQP7x7o4FDdgb9GvPDoFzI
From (redirected): https://drive.google.com/uc?id=1B0qWNzQZQmfQP7x7o4FDdgb9GvPDoFzI&confirm=t&uuid=4a711dcd-b1c0-492c-a48b-21b997382f72
To: /content/hw3-data-release.tar.gz
100% 387M/387M [00:06<00:00, 62.3MB/s]
mkdir: cannot create directory ‘../dataset’: File exists
mv: cannot move 'test_release/' to '../dataset/test_release': Directory not empty
mv: cannot move 'train/' to '../dataset/train': Directory not empty


In [50]:
class MedicalDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = self._load_samples()

        self.train_coco_path = os.path.join(pathlib.Path(root_dir).parent, 'train_coco.json')
        if not os.path.exists(self.train_coco_path):
            self.generate_coco(self.train_coco_path)
        self.train_coco = COCO(self.train_coco_path)
        self.num_classes = len(self.train_coco.loadCats(self.train_coco.getCatIds()))

    def _load_samples(self):
        samples = []
        for img_dir in os.listdir(self.root):
            tmp_dir = os.path.join(self.root, img_dir)

            if not self.is_test:
                img_path = os.path.join(tmp_dir, 'image.tif')

                mask_paths = [
                    entry.name for entry in pathlib.Path(tmp_dir).iterdir()
                    if entry.name.startswith("class") and entry.is_file()
                ]

                samples.append({'image': img_path, 'masks': mask_paths})
            else:
                test_img_json_path = os.path.join(pathlib.Path(self.root).parent, 'test_image_name_to_ids.json')
                with open(test_img_json_path, 'r') as f:
                    samples = json.load(f)

                # for idx in range(len(samples)):
                #     samples[idx]['file_name'] = os.path.join(self.root, samples[idx]['file_name'])
        return samples

    def mask_to_polygons(self, mask, epsilon=1.0):
        contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        polygons = []
        for contour in contours:
            if len(contour) > 2:
                poly = contour.reshape(-1).tolist()
                if len(poly) > 4: #Ensures valid polygon
                    polygons.append(poly)
        return polygons

    def generate_coco(self, output_dir, train=True):
        annotations = []
        images = []
        categories = []
        all_labels = []
        ann_id = 0

        for img_id, sample in enumerate(self.samples):
            print(f'({img_id}/{len(self.samples)})')
            img_path, mask_paths = sample['image'], sample['masks']
            img = cv2.imread(img_path)
            masks = [cv2.imread(os.path.join(pathlib.Path(img_path).parent, mask_path), cv2.IMREAD_UNCHANGED) for mask_path in mask_paths]

            images.append({
                "id": img_id,
                "file_name": img_path,
                "height": img.shape[0],
                "width": img.shape[1]
            })

            for mask in masks:
                unique_values = np.unique(mask)
                all_labels.append(unique_values)
                for value in unique_values:
                    if value == 0:  # Ignore background
                        continue

                    object_mask = (mask == value).astype(np.uint8) * 255
                    polygons = self.mask_to_polygons(object_mask)

                    for poly in polygons:
                        ann_id += 1
                        annotations.append({
                            "id": ann_id,
                            "image_id": img_id,
                            "category_id": 1,  # Only one category: Nuclei
                            "segmentation": [poly],
                            "area": cv2.contourArea(np.array(poly).reshape(-1, 2)),
                            "bbox": list(cv2.boundingRect(np.array(poly).reshape(-1, 2))),
                            "iscrowd": 0
                        })

        all_labels = np.unique(np.concatenate(all_labels).tolist())

        for idx, label in enumerate(all_labels):
            categories.append({"id": idx+1, "name": int(label)})

        coco_input = {
            "images": images,
            "annotations": annotations,
            "categories": categories
        }

        print(f'Saving train coco json')
        # if train:
        with open(output_dir, 'w') as f:
            json.dump(coco_input, f)
        # else:
        #     with open(os.path.join(output_dir, 'instances_val2017.json'), 'w') as f:
        #         json.dump(coco_input, f)



    # def pickout_label_in_mask(self, mask):
    #     all_labels_in_mask = np.unique(mask)

    #     res = {}

    #     masks = []
    #     labels = []
    #     for label in all_labels_in_mask:
    #         if label > self.num_classes:
    #             self.num_classes = label
    #         if label == 0:
    #             continue

    #         tmp = np.zeros_like(mask)
    #         tmp[mask == label] = 1
    #         masks.append(tmp.astype(np.uint8))
    #         labels.append(label)

    #     res['mask'] = masks
    #     res['label'] = labels

    #     return res


    # def generate_target(self, sample):
    #     target = {}

    #     # for sample in samples:
    #     image_path = sample['image']
    #     mask_dict = sample['masks']

    #     boxes = []
    #     labels = []
    #     masks = []
    #     for mask in mask_dict:
    #         mask_path = os.path.join(pathlib.Path(image_path).parent, mask)
    #         mask_image = imread(mask_path)
    #         mask_with_label = self.pickout_label_in_mask(mask_image)

    #         for i in range(len(mask_with_label['mask'])):
    #             mask = mask_with_label['mask'][i]
    #             category = mask_with_label['label'][i]

    #             if np.sum(mask) > 0:
    #                 mask = mask.astype(np.uint8)
    #                 contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #                 for contour in contours:
    #                     if len(contour) > 2:
    #                         masks.append(mask)
    #                         labels.append(category)
    #                         boxes.append(cv2.boundingRect(contour))

    #     target['boxes'] = torch.tensor(np.array(boxes), dtype=torch.float32)
    #     target['labels'] = torch.tensor(np.array(labels), dtype=torch.int64)
    #     target['masks'] = torch.tensor(np.array(masks), dtype=torch.uint8)

    #     return target

    # def __getitem__(self, idx):
    #     sample = self.samples[idx]

    #     if not self.is_test:
    #         image = torch.tensor(cv2.imread(sample['image']), dtype=torch.float32).permute(2, 0, 1)#.astype(np.float32) / 255.0
    #         target = self.generate_target(sample)

    #         target['boxes'] = box_convert(target['boxes'], in_fmt='xywh', out_fmt='xyxy')

    #         return self.transform(image), target
    #     else:

    #         return 1, 0

    def poly2mask(self, segmentation, img_size):
        """
        多邊形標註轉二值掩碼
        :param segmentation: COCO格式的多邊形坐標列表 [[x1,y1,x2,y2,...]]
        :param img_size: 目標圖像尺寸 (height, width)
        """
        # 自動檢測標註類型
        if isinstance(segmentation, dict):
            # 處理RLE格式
            return coco_mask.decode(segmentation)
        else:
            # 處理多邊形格式
            rle = coco_mask.frPyObjects(segmentation, img_size[0], img_size[1])
            return coco_mask.decode(rle)

    def __getitem__(self, index):
        sample = self.samples[index]

        if not self.is_test:
            img_ids = self.train_coco.getImgIds(imgIds=index)
            img_info = self.train_coco.loadImgs(img_ids)
            # image = cv2.imread(img_info[0]['file_name']) / 255.0
            image = Image.open(img_info[0]['file_name']).convert("RGB")
            image = self.transform(image)
            img_size = [img_info[0]['height'], img_info[0]['width']]


            boxes = []
            masks = []
            labels = []
            ann_ids = self.train_coco.getAnnIds(imgIds=img_ids)
            annotations = self.train_coco.loadAnns(ann_ids)
            for ann in annotations:
                boxes.append(ann['bbox'])
                tmp_mask = self.poly2mask(ann['segmentation'], img_size).squeeze()
                # mask_ = F.interpolate(
                #     tmp_mask,
                #     size=(224, 224),
                #     mode='nearest-exact'  # PyTorch 1.10+ 專用選項
                # )
                mask_ = cv2.resize(
                    tmp_mask,
                    (224, 224),
                    interpolation=cv2.INTER_NEAREST_EXACT  # 精確最近鄰算法
                )
                masks.append(mask_)
                labels.append(ann["category_id"])

            boxes = self.resize_box(boxes, img_size, target_size=[224,224])
            boxes = box_convert(torch.tensor(boxes, dtype=torch.float32), in_fmt='xywh', out_fmt='xyxy')
            masks = torch.as_tensor(np.array(masks), dtype=torch.bool)

            target = {'boxes': torch.as_tensor(boxes, dtype=torch.float32),
                      'masks': masks,
                      'labels': torch.as_tensor(np.array(labels), dtype=torch.int64)}

            return image, target
        else:

            return 1, 0

    def resize_box(self, boxes, orig_size, target_size):
        # Eat xywh
        scale_w = target_size[1] / orig_size[1]
        scale_h = target_size[0] / orig_size[0]

        for box in boxes:
            box[0] *= scale_w  # x
            box[1] *= scale_h  # y
            box[2] *= scale_w  # w
            box[3] *= scale_h  # h

        return boxes

    def __len__(self):
        return len(self.samples)

In [16]:
project_root = '..'
train_dir = os.path.join(project_root, 'dataset/train')
test_dir = os.path.join(project_root, 'dataset/test_release')

In [17]:
# train_coco_path = f'/home/bhg/visual_dl/lab3/dataset'
# train_set = MedicalDataset(root_dir=train_dir)
# train_set.generate_coco(train_coco_path)

In [51]:
train_transform=T.Compose([
    T.ToTensor(),
    T.Resize(size=[224,224], antialias=True),
    # T.CenterCrop(size=224),
    # T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_set = MedicalDataset(root_dir=train_dir, transform=train_transform)
img, target = train_set[1]
print(f"box: {target['boxes'].shape}")
print(f"mask: {target['masks'].shape}")
print(f"label: {target['labels'].shape}")
print(target['boxes'][0])

print(img, img.shape)

loading annotations into memory...
Done (t=0.70s)
creating index...
index created!
box: torch.Size([86, 4])
mask: torch.Size([86, 224, 224])
label: torch.Size([86])
tensor([ 16.8142, 104.8320,  35.6234, 147.3920])
tensor([[[0.7283, 0.5805, 0.5480,  ..., 0.5469, 0.5975, 0.5980],
         [0.7332, 0.6403, 0.5805,  ..., 0.5863, 0.6309, 0.6200],
         [0.7627, 0.6954, 0.6048,  ..., 0.6524, 0.6922, 0.5968],
         ...,
         [0.3114, 0.2937, 0.2715,  ..., 0.3510, 0.4343, 0.6885],
         [0.3014, 0.2804, 0.2598,  ..., 0.3649, 0.4413, 0.6135],
         [0.2940, 0.2680, 0.3035,  ..., 0.3818, 0.5199, 0.6588]],

        [[0.6085, 0.4161, 0.3751,  ..., 0.3781, 0.4364, 0.4375],
         [0.5894, 0.4423, 0.3612,  ..., 0.4151, 0.4654, 0.4284],
         [0.6923, 0.4871, 0.3765,  ..., 0.4883, 0.5478, 0.4395],
         ...,
         [0.2613, 0.2613, 0.2400,  ..., 0.1632, 0.2396, 0.5874],
         [0.2671, 0.2413, 0.2507,  ..., 0.1689, 0.2120, 0.4515],
         [0.2613, 0.2429, 0.2717,  ..., 0

In [52]:
# train_transform = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.3),
#     A.Rotate(limit=15, p=0.4),
#     A.CLAHE(p=0.5),
#     A.GridDistortion(p=0.2),
#     A.RandomBrightnessContrast(p=0.3)
# ], additional_targets={'mask': 'mask'})
train_transform=T.Compose([
    T.ToTensor(),
    T.Resize(size=[224, 224], antialias=True),
    # T.CenterCrop(size=224),
    # T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_set = MedicalDataset(root_dir=train_dir, transform=train_transform)
# test_set = MedicalDataset(root_dir=test_dir, is_test=True)

# def custom_collate(batch):
#     images = []
#     targets = []

#     for img, target in batch:
#         images.append(img)
#         targets.append({
#             'boxes': target['boxes'],
#             'labels': target['labels'],
#             'masks': target['masks']
#         })

#     images = torch.stack(images, dim=0)
#     return images, targets


loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


In [56]:
# MAX_BOXES_PER_BATCH = 5  # 依GPU記憶體調整

# def custom_collate(batch):
#     images = []
#     targets = []
#     residual_boxes = []  # 暫存未處理的boxes

#     for img, target in batch:
#         # 合併殘留框與新框
#         all_boxes = residual_boxes + target['boxes']

#         # 切割可用區塊
#         use_boxes = all_boxes[:MAX_BOXES_PER_BATCH]
#         residual_boxes = all_boxes[MAX_BOXES_PER_BATCH:]

#         # 僅在有可用框時加入當前批次
#         if len(use_boxes) > 0:
#             images.append(img)
#             targets.append({
#                 'boxes': use_boxes,
#                 'labels': target['labels'][:len(use_boxes)],
#                 'masks': target['masks'][:len(use_boxes)]
#             })

#     # 遞歸處理殘留框
#     if residual_boxes:
#         dummy_img = torch.zeros_like(img)  # 填充虛擬數據
#         return custom_collate([(dummy_img, {'boxes': residual_boxes})] + batch)

#     images = torch.stack(images)
#     return images, targets
def custom_collate(batch):
    images = []
    targets = []

    for img, target in batch:
        images.append(img)
        targets.append({
            'boxes': target['boxes'],
            'labels': target['labels'],
            'masks': target['masks']
        })

    images = torch.stack(images, dim=0)
    return images, targets


BATCH_SIZE = 2
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=False, collate_fn=custom_collate)

In [ ]:
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
model.to('cuda')
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 10

from tqdm import tqdm


for epoch in tqdm(range(num_epochs), desc="Epochs"):
    print(f'a')
    model.train()
    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for images, targets in bar:
        # print('b')
        images = [image.to('cuda') for image in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        # import torch.nn.functional as F
        # images = F.interpolate(
        #     images,
        #     size=224,
        #     mode='bilinear',
        #     align_corners=False
        # )

        # targets['masks'] = F.interpolate(
        #     targets['masks'],
        #     size=224,
        #     mode='nearest'
        # )

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        bar.set_postfix(loss=losses.detach().cpu().item())
        bar.update()
    lr_scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses.item()}")



Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

a



Epoch 1/10:  90%|████████▉ | 94/105 [00:38<00:05,  1.95it/s, loss=1.26]

In [ ]:
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
model = maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
model.to('cuda')
model.eval()

from tqdm import tqdm
from torchvision.ops import box_convert

bar = tqdm(train_loader, desc="Inference", total=len(train_loader))
for images, targets in bar:
    images = list(image.to('cuda') for image in images)
    targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]

    with torch.no_grad():
        predictions = model(images)

    for i, prediction in enumerate(predictions):
        boxes = prediction['boxes'].cpu().numpy()
        masks = prediction['masks'].cpu().numpy()
        labels = prediction['labels'].cpu().numpy()

        # Process the predictions as needed
        print(f"Image {i}:")
        print("Boxes:", boxes)
        print("Masks:", masks)
        print("Labels:", labels)

    bar.update()

Inference:   0%|          | 0/53 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model = torchvision.models.get_model(
        args.model, weights=args.weights, weights_backbone=args.weights_backbone, num_classes=num_classes, **kwargs
    )
model.roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=len(class_names),bias=True)
model.roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=len(class_names)*4,bias=True)
model.roi_heads.mask_predictor.mask_fcn_logits = nn.Conv2d(256, len(class_names),kernel_size=(1,1),stride=(1,1))

model.to(device)

In [ ]:
class EnhancedMaskRCNN(MaskRCNN):
    def __init__(self, backbone, num_classes=None, **kwargs):
        super().__init__(backbone, num_classes, **kwargs)
        # 添加邊界感知分支
        self.boundary_head = self._build_boundary_head()

    def _build_boundary_head(self):
        layers = [
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 1, 1)  # 邊界檢測輸出
        ]
        return torch.nn.Sequential(*layers)

    def forward(self, images, targets=None):
        # 原始輸出
        outputs = super().forward(images, targets)

        # 邊界檢測分支
        if self.training and targets is not None:
            boundary_maps = self.boundary_head(outputs['features'])
            outputs['boundary_loss'] = self.compute_boundary_loss(boundary_maps, targets)

        return outputs

def create_model(num_classes=5, pretrained=True):
    backbone = torchvision.models.resnet50(weights=ResNet50_Weights.DEFAULT)
    # backbone = torchvision.models._utils.IntermediateLayerGetter(
    #     backbone, return_layers={'layer4': 'out'}
    # )
    # backbone = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

    model = EnhancedMaskRCNN(
        backbone,
        num_classes=num_classes,
        min_size=512,
        max_size=512
    )

    # 修改分類頭
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 修改mask頭
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, 256, num_classes
    )

    return model


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = create_model().to(device)
params = [p for p in model.parameters() if p.requires_grad]

optimizer = SGD(params, lr=0.003, momentum=0.9, weight_decay=0.0005)
lr_sched = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 混合損失函數
def hybrid_loss(pred_masks, gt_masks, boundaries):
    mask_loss = torch.nn.functional.binary_cross_entropy_with_logits(pred_masks, gt_masks)
    boundary_loss = torch.nn.functional.mse_loss(pred_masks, boundaries)
    return mask_loss + 0.3 * boundary_loss

NUM_EPOCHS = 30

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0

    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [{'masks': t.to(device)} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    lr_sched.step()
    print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.4f}")



AttributeError: 'ResNet' object has no attribute 'out_channels'

In [ ]:
def masks_to_coco(results, image_ids):
    coco_results = []
    for img_id, output in zip(image_ids, results):
        for score, mask, label in zip(output['scores'], output['masks'], output['labels']):
            rle = binary_mask_to_rle(mask)
            coco_results.append({
                "image_id": img_id,
                "category_id": label.item(),
                "segmentation": rle,
                "score": score.item()
            })
    return coco_results

def binary_mask_to_rle(mask):
    # RLE編碼實現
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return {'size': list(mask.shape[-2:]), 'counts': runs.tolist()}


In [ ]:
model.eval()
test_loader = DataLoader(test_set, batch_size=2, shuffle=False)

results = []
with torch.no_grad():
    for batch in test_loader:
        outputs = model(batch.to(device))
        results.extend(outputs)

# 生成最終提交文件
with open('test-results.json', 'w') as f:
    json.dump(masks_to_coco(results, test_set.image_ids), f)

print("Submission file generated!")